<a href="https://colab.research.google.com/github/Sunaina-Imran/Medical_insurance_premium_api/blob/main/Medical_insurance_premium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendatasets


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/tejashvi14/medical-insurance-premium-prediction")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: KGAT_e401d941b43fdf73f703530135cdbb0e
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/tejashvi14/medical-insurance-premium-prediction


100%|██████████| 7.08k/7.08k [00:00<00:00, 3.48MB/s]

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

data = pd.read_csv("/content/medical-insurance-premium-prediction/Medicalpremium.csv")

In [ ]:
data.sample(5)

,Age,Diabetes,BloodPressureProblems,AnyTransplants,AnyChronicDiseases,Height,Weight,KnownAllergies,HistoryOfCancerInFamily,NumberOfMajorSurgeries,PremiumPrice
960,23,0,0,0,0,175,73,0,0,0,15000
532,49,0,0,0,0,178,81,1,0,1,29000
230,63,1,1,0,0,167,58,1,1,1,25000
164,29,0,0,0,0,180,76,0,0,0,15000
421,36,0,0,0,0,165,72,0,0,0,23000


In [ ]:
data_feat = data.copy()

In [ ]:
# Feature 1: BMI
data_feat["BMI"] = data_feat["Weight"] / ((data_feat["Height"]/100) ** 2)

In [ ]:
# Feature 2: Age Group
def age_group(Age):
    if Age < 30:
        return "Young"
    elif Age < 45:
        return "Adult"
    elif Age < 60:
        return "Middle_Age"
    return "Senior"

In [ ]:
data_feat["Age_group"] = data_feat["Age"].apply(age_group)

In [ ]:
data_feat.drop(
    columns=['Age', 'Weight', 'Height', 'AnyChronicDiseases']
)[['Age_group','BMI' , 'Diabetes', 'BloodPressureProblems',
   'AnyTransplants', 'KnownAllergies',
   'HistoryOfCancerInFamily', 'NumberOfMajorSurgeries',
   'PremiumPrice']]



,Age_group,BMI,Diabetes,BloodPressureProblems,AnyTransplants,KnownAllergies,HistoryOfCancerInFamily,NumberOfMajorSurgeries,PremiumPrice
0,Middle_Age,23.725286,0,0,0,0,0,0,25000
1,Senior,22.530864,1,0,0,0,0,0,29000
2,Adult,23.634033,1,1,0,0,0,1,23000
3,Middle_Age,27.770313,1,1,0,0,0,2,28000
4,Adult,31.934969,0,0,0,0,0,1,23000
...,...,...,...,...,...,...,...,...,...
981,Young,23.458562,0,0,0,0,0,0,15000
982,Senior,29.903029,1,1,0,0,0,3,28000
983,Middle_Age,29.552549,0,1,0,0,0,1,29000
984,Middle_Age,29.242109,1,1,0,1,0,1,39000


In [ ]:
# Select features and target
X = data_feat[['Age_group','BMI','Diabetes','BloodPressureProblems','AnyTransplants','KnownAllergies',
               'HistoryOfCancerInFamily','NumberOfMajorSurgeries']]

y = data_feat['PremiumPrice']

In [ ]:
# Define categorical and numeric features
categorical_features = ['Age_group']
numerical_features = ['BMI','Diabetes','BloodPressureProblems',
                      'AnyTransplants','KnownAllergies',
                      'HistoryOfCancerInFamily','NumberOfMajorSurgeries']


In [ ]:
# Create coloum transformer
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop='first'), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

In [ ]:
# Create pipeline
model_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestRegressor(
        n_estimators=400,
        max_depth=12,
        min_samples_split=5,
        random_state=42
    ))
])

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['Age_group']),
                                                 ('num', 'passthrough',
                                                  ['BMI', 'Diabetes',
                                                   'BloodPressureProblems',
                                                   'AnyTransplants',
                                                   'KnownAllergies',
                                                   'HistoryOfCancerInFamily',
                                                   'NumberOfMajorSurgeries'])])),
                ('model',
                 RandomForestRegressor(max_depth=12, min_samples_split=5,
                                       n_estimators=400, random_state=42))])

In [ ]:
# Train
model_pipeline.fit(X_train, y_train)

# Predict
y_pred = model_pipeline.predict(X_test)

In [ ]:
# Evaluate
print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R2 Score: 0.7864284795578187
MAE: 2055.537937237415
RMSE: 3017.8279983515536


In [ ]:
X_test.sample(5)

,Age_group,BMI,Diabetes,BloodPressureProblems,AnyTransplants,KnownAllergies,HistoryOfCancerInFamily,NumberOfMajorSurgeries
23,Adult,27.102041,0,1,0,0,0,1
137,Senior,21.220009,1,1,0,0,0,2
916,Senior,24.840980,1,1,0,0,0,1
707,Middle_Age,18.612245,0,1,0,0,0,0
792,Young,20.177149,1,0,0,0,0,0


In [ ]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = "model.pkl"

with open(pickle_model_path, "wb") as f:
    pickle.dump(model_pipeline, f)
